# How to train YOLOv3 using Darknet on Colab notebook and optimize the VM runtime load times

## Welcome!

This Colab notebook will show you how to:

* Train a **Yolo v3** model using **Darknet** using the Colab **12GB-RAM GPU**.
* Turn Colab notebooks into an effective tool to work on real projects. Dealing with the handicap of a runtime that will **blow up every 12 hours** into the space!
  * Working directly from the files on your computer.
  * Configure your notebook to install everything you need and start training in about a minute (Tested using 550MB dataset).
  * Receive your trained weights directly on your computer during the training. While the notebook is training you can check how it is going using your trained weights in your computer.


#### This notebook is part of the post [How to train YOLOv3 using Darknet framework and optimize the VM runtime load times](http://blog.ibanyez.info/blogs/coding/20190410-run-a-google-colab-notebook-to-train-yolov3-using-darknet-in/) I encourage you to visit! You will find a deeper explanation about Google Colab, the goods and the limitations of this great tool.


### These are the steps we'll follow:

* Configure Google Drive and map as network  _Drive_.
* Some utils to help to do some tasks.
* Configure the pre-requisites on the runtime.
  * Check the CUDA installation on the runtime VM.
  * Install cuDNN.
  * Clone and compile Darknet. We'll use a repo based on [AlexeyAB's Darknet repo](https://github.com/AlexeyAB/darknet/). I applied  some changes to make possible to load files from `/My Drive/` and reduced the number of logs on console to speed up the notebook.
  * We'll check that everything works great.
* Explained how to manage your YOLO files on your computer and it will be used transparently from this notebook.

> _**NOTE:** Cells with an annotation **`# Not Necessary cell`**. Can be removed without having any impact. They are only explanatory content._

### Without further ado, let's start!


Vou utilizar nesse projeto a YoloV5


Para iniciar, vamos baixar o código-fonte do YOLOv5 do repositório oficial no GitHub

In [ ]:
!git clone https://github.com/ultralytics/yolov5.git


In [ ]:
%cd yolov5
!pip install -r requirements.txt


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Ajustando o dataSet no meu Drive

In [ ]:
# Caminho onde você fez o upload
path_to_drive = '/content/drive/MyDrive/COCO/'

# Descompactando as imagens
!unzip -q {path_to_drive}val2017.zip -d {path_to_drive}val2017/

# Descompactando as anotações
!unzip -q {path_to_drive}annotations_trainval2017.zip -d {path_to_drive}annotations/


In [ ]:
# Movendo imagens para a pasta correta
!mv /content/drive/MyDrive/COCO/val2017/val2017/* /content/drive/MyDrive/COCO/val2017/

# Movendo anotações para a pasta correta
!mv /content/drive/MyDrive/COCO/annotations/annotations/* /content/drive/MyDrive/COCO/annotations/


In [ ]:
# Verificando imagens de validação
!ls /content/drive/MyDrive/COCO/val2017/

# Verificando anotações
!ls /content/drive/MyDrive/COCO/annotations/


In [ ]:
import os
import json

In [ ]:

# Definindo os caminhos corretos
coco_images_path = "/content/drive/MyDrive/COCO/val2017/"
coco_annotations_path = "/content/drive/MyDrive/COCO/annotations/instances_val2017.json"

# Verificando se o arquivo de anotações existe
if os.path.exists(coco_annotations_path):
    with open(coco_annotations_path, "r") as f:
        coco_data = json.load(f)
    print("Anotações carregadas com sucesso!")
    print("Número de imagens anotadas:", len(coco_data["images"]))
    print("Número de categorias:", len(coco_data["categories"]))
else:
    print("Arquivo de anotações não encontrado.")


In [ ]:
import random
import matplotlib.pyplot as plt
import cv2
from google.colab.patches import cv2_imshow

In [ ]:
#Escolhendo uma imagem do dataset
random_image_info = random.choice(coco_data["images"])
image_id = random_image_info["id"]
image_filename = random_image_info["file_name"]

#Caminho
image_path = os.path.join(coco_images_path, image_filename)

#Carregando a imagem com OpenCV
image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) #Convertendo para RGB

#Exibir a imagem
plt.figure(figsize=(8,8))
plt.imshow(image)
plt.axis("off")
plt.title(f"Imagem ID: {image_id}")
plt.show()

In [ ]:
import shutil
import json
import os
from pathlib import Path

# Diretórios
annotations_file = '/content/drive/MyDrive/COCO/annotations/instances_val2017.json'
images_source_dir = '/content/drive/MyDrive/COCO/val2017'
images_dest_dir = '/content/drive/MyDrive/COCO/yolo_images'

# Mapear classes para os índices do YOLO
class_map = {
    1: 0,  # Pessoa -> Classe 0
    3: 1   # Carro -> Classe 1
}

# Criar o diretório de destino para as imagens
Path(images_dest_dir).mkdir(parents=True, exist_ok=True)

# Função para mover imagens de "pessoa" e "carro"
def move_images_by_class():
    with open(annotations_file, 'r') as f:
        coco_data = json.load(f)

    # Iterar sobre as anotações
    for annotation in coco_data['annotations']:
        if annotation['category_id'] in class_map:  # Verificar se a classe é "pessoa" ou "carro"
            image_id = annotation['image_id']
            # Encontrar a imagem correspondente
            image_info = next((img for img in coco_data['images'] if img['id'] == image_id), None)
            if image_info:
                # Caminho da imagem
                image_path = os.path.join(images_source_dir, image_info['file_name'])
                dest_path = os.path.join(images_dest_dir, image_info['file_name'])

                # Copiar imagem para o diretório de destino
                shutil.copy(image_path, dest_path)

# Chamar a função para mover as imagens
move_images_by_class()


In [ ]:
import os

images_dest_dir = '/content/drive/MyDrive/COCO/yolo_images'
print("Imagens copiadas:", os.listdir(images_dest_dir)[:10])  # Mostra as 10 primeiras imagens


In [ ]:
print("Total de imagens copiadas:", len(os.listdir(images_dest_dir)))


In [ ]:
import json

annotations_file = '/content/drive/MyDrive/COCO/annotations/instances_val2017.json'

with open(annotations_file, 'r') as f:
    coco_data = json.load(f)

category_ids = set(annotation['category_id'] for annotation in coco_data['annotations'])
print("IDs das categorias disponíveis no dataset:", category_ids)


Agora vamos gerar os arquivos de anotações .txt para cada imagem correpondente, para treinar a rede Yolo.


In [ ]:
import os
import json
from pathlib import Path

# Diretórios
annotations_file = '/content/drive/MyDrive/COCO/annotations/instances_val2017.json'
images_dest_dir = '/content/drive/MyDrive/COCO/yolo_images'
labels_dest_dir = '/content/drive/MyDrive/COCO/yolo_labels'

# Criar o diretório de destino para os arquivos de anotação
Path(labels_dest_dir).mkdir(parents=True, exist_ok=True)

# Mapear classes para os índices do YOLO
class_map = {
    1: 0,  # Pessoa -> Classe 0
    3: 1   # Carro -> Classe 1
}

# Abrir o arquivo JSON de anotações
with open(annotations_file, 'r') as f:
    coco_data = json.load(f)

# Criar um dicionário para armazenar as anotações por imagem
annotations_by_image = {}
for annotation in coco_data['annotations']:
    if annotation['category_id'] in class_map:
        image_id = annotation['image_id']
        image_filename = f"{image_id:012d}.jpg"

        bbox = annotation['bbox']  # [x_min, y_min, width, height]
        x_min, y_min, width, height = bbox

        # Encontrar informações da imagem
        image_info = next((img for img in coco_data['images'] if img['id'] == image_id), None)
        if not image_info:
            continue

        img_width, img_height = image_info['width'], image_info['height']

        # Converter para formato YOLO
        x_center = (x_min + width / 2) / img_width
        y_center = (y_min + height / 2) / img_height
        width /= img_width
        height /= img_height

        # Adicionar anotação ao dicionário
        if image_filename not in annotations_by_image:
            annotations_by_image[image_filename] = []

        annotations_by_image[image_filename].append(
            f"{class_map[annotation['category_id']]} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}"
        )

# Criar arquivos de anotação para cada imagem
for image_filename, annotations in annotations_by_image.items():
    label_filename = os.path.join(labels_dest_dir, image_filename.replace('.jpg', '.txt'))
    with open(label_filename, 'w') as label_file:
        label_file.write('\n'.join(annotations))

print(f"Arquivos de anotação gerados: {len(annotations_by_image)}")


Criando o diretório yolo_dataset

In [ ]:
import os
from pathlib import Path

# Criar o diretório, se não existir
data_dir = "/content/drive/MyDrive/COCO/yolo_dataset"
Path(data_dir).mkdir(parents=True, exist_ok=True)

# Conteúdo do arquivo YAML
data_yaml = """train: /content/drive/MyDrive/COCO/yolo_dataset/train/images
val: /content/drive/MyDrive/COCO/yolo_dataset/val/images

nc: 2
names: ["pessoa", "carro"]
"""

# Criar o arquivo data.yaml
yaml_path = os.path.join(data_dir, "data.yaml")
with open(yaml_path, "w") as f:
    f.write(data_yaml)

print(f"Arquivo 'data.yaml' criado com sucesso em: {yaml_path}")


Gerar o .YAML

In [ ]:
data_yaml = """train: /content/drive/MyDrive/COCO/yolo_dataset/train/images
val: /content/drive/MyDrive/COCO/yolo_dataset/val/images

nc: 2
names: ["pessoa", "carro"]
"""

# Criar o arquivo
with open("/content/drive/MyDrive/COCO/yolo_dataset/data.yaml", "w") as f:
    f.write(data_yaml)

print("Arquivo 'data.yaml' criado com sucesso!")


Preparar a estrutura do diretório e separar as imagens para o treinamento


In [ ]:
import os
from pathlib import Path
import shutil
import random

# Definir os caminhos
base_dir = "/content/drive/MyDrive/COCO/yolo_dataset"
train_images_dir = os.path.join(base_dir, "train/images")
train_labels_dir = os.path.join(base_dir, "train/labels")
val_images_dir = os.path.join(base_dir, "val/images")
val_labels_dir = os.path.join(base_dir, "val/labels")

# Criar os diretórios
for dir_path in [train_images_dir, train_labels_dir, val_images_dir, val_labels_dir]:
    Path(dir_path).mkdir(parents=True, exist_ok=True)

# Listar todas as imagens e misturar aleatoriamente
all_images = os.listdir("/content/drive/MyDrive/COCO/yolo_images")
random.shuffle(all_images)

# Separar 85% para treino e 15% para validação
train_split = int(len(all_images) * 0.85)
train_files = all_images[:train_split]
val_files = all_images[train_split:]

# Função para mover imagens e anotações
def move_files(files, dest_img_dir, dest_label_dir):
    for file_name in files:
        img_path = f"/content/drive/MyDrive/COCO/yolo_images/{file_name}"
        label_path = img_path.replace(".jpg", ".txt").replace("yolo_images", "yolo_labels")

        if os.path.exists(img_path):
            shutil.move(img_path, os.path.join(dest_img_dir, file_name))

        if os.path.exists(label_path):
            shutil.move(label_path, os.path.join(dest_label_dir, os.path.basename(label_path)))

# Mover arquivos para treino e validação
move_files(train_files, train_images_dir, train_labels_dir)
move_files(val_files, val_images_dir, val_labels_dir)

print("Divisão de treino e validação concluída!")


Definir o modelo YOLO


In [ ]:
#Verificar se há GPU disponível no colab para usar
import torch
print(torch.cuda.is_available())  # Se for True, use --device 0


Vamos rodar o modelo. Aqui eu precisei ajustar alguns parametros do treinamento para facilita-lo, devido ao desempenho.

In [ ]:
# Desativar Weights & Biases e rodar o treinamento
%env WANDB_DISABLED=True
!python train.py --img 200 --batch 8 --epochs 8 --data /content/drive/MyDrive/COCO/yolo_dataset/data.yaml --weights yolov5s.pt --device cpu --nosave --cache  --name my_yolo --project /content/drive/MyDrive/COCO/yolo_training --freeze 10



Modelo treinado com o Transfer Learning. Agora vamos validar testando uma imagem.

In [ ]:
!python detect.py --weights /content/drive/MyDrive/COCO/yolo_training/my_yolo7/weights/best.pt --img 320 --conf 0.5 --source /content/drive/MyDrive/COCO/yolo_images/teste1.jpg --device cpu


Vamos utilizar uma biblioteca para visualizar a imagem teste e validar se houve a detecção!

In [ ]:
from IPython.display import display
from PIL import Image

image_path = "runs/detect/exp4/teste1.jpg"  # Ajuste caso o nome seja diferente
display(Image.open(image_path))
